In [ ]:
from ultralytics import YOLO
from torch import nn
import torch

my_data = "./data.yaml"
model = YOLO("yolo11n.pt")





Sequential(
  (0): Conv(
    (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (1): Conv(
    (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (2): C3k2(
    (cv1): Conv(
      (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (cv2): Conv(
      (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (m): ModuleList(
      (0): Bottleneck(
        (cv1): Conv(
       

In [5]:
import torch
import torch.nn as nn
from torchinfo import summary
from ultralytics import YOLO

# 모델 로드
model = YOLO("yolo11n.pt")

# 백본 레이어 추출 (neck 레이어까지 제거)
# 예를 들어, neck 레이어가 10번째 레이어라면, 그 이후의 레이어들을 제거합니다.
backbone_layers = list(model.model.model.children())[:10]
backbone = nn.Sequential(*backbone_layers)

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
backbone.to(device)


# 더미 입력 생성
input_tensor = torch.randn(1, 3, 640, 640).to(device)

# 모델 순전파 실행
with torch.no_grad():
    try:
        output = backbone(input_tensor)
        print(f'백본 출력 크기: {output.shape}')
    except TypeError as e:
        print(f'오류 발생: {e}')

# 모델 서머리 출력 (배치 크기를 포함)
summary(backbone, input_size=(1, 3, 640, 640))

백본 출력 크기: torch.Size([1, 256, 20, 20])


Layer (type:depth-idx)                                  Output Shape              Param #
Sequential                                              [1, 256, 20, 20]          --
├─Conv: 1-1                                             [1, 16, 320, 320]         --
│    └─Conv2d: 2-1                                      [1, 16, 320, 320]         (432)
│    └─BatchNorm2d: 2-2                                 [1, 16, 320, 320]         (32)
├─SPPF: 1-62                                            --                        (recursive)
│    └─Conv: 2-69                                       --                        (recursive)
│    │    └─SiLU: 3-1                                   [1, 16, 320, 320]         --
├─Conv: 1-3                                             [1, 32, 160, 160]         --
│    └─Conv2d: 2-4                                      [1, 32, 160, 160]         (4,608)
│    └─BatchNorm2d: 2-5                                 [1, 32, 160, 160]         (64)
├─SPPF: 1-62                  

In [ ]:
model.model.head = CustomHead(nc=80, ch=[256])
model.loss_fn = CustomLoss()